In [4]:
!pip install -U sentence-transformers

In [5]:
import pandas as pd
import numpy as np

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

model_name = "MoritzLaurer/ernie-m-large-mnli-xnli"
tokenizer = AutoTokenizer.from_pretrained(model_name)
nli_model = AutoModelForSequenceClassification.from_pretrained(model_name)

KeyboardInterrupt: ignored

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
data = pd.read_csv('/content/drive/MyDrive/dataset/public_test.csv')

In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize

nltk.download('punkt')
nltk.download('stopwords')

def preprocess_text(text,max_words = 150):
    if isinstance(text, str):
        # Tách câu
        sentences = sent_tokenize(text)

        preprocessed_sentences = []

        for sentence in sentences:
            # Chuyển thành chữ thường
            preprocessed_sentence = sentence.lower()

            preprocessed_sentences.append(preprocessed_sentence) #list

        preprocessed_passages = []
        preprocessed_passage = ''
        count = 0
        for sentence in preprocessed_sentences:
            words = word_tokenize(sentence)
            count += len(words)
            if count > max_words: #sentence chuyen qua passage sau
                preprocessed_passages.append(preprocessed_passage)
                count = len(words)
                preprocessed_passage = sentence
            else:
                preprocessed_passage += sentence

        preprocessed_passages.append(preprocessed_passage)
        return preprocessed_passages

    else:
        return [text]

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
data = data[3600:]
data

,index,context,claim
3600,32560,"Năm 2015, sau khi tốt nghiệp trường THPT chuyê...",an đã theo học ở mỹ và học song bằng theo lời ...
3601,46212,"Sở hữu tiết trời mát mẻ quanh năm, cảnh quan t...","Đại diện Sun World Ba Na Hills cho biết, Hàn Q..."
3602,29704,"Ngày 13/3, PGS.TS Phạm Cẩm Phương, Giám đốc Tr...",Từ 10-15% trường hợp ung thư ở Việt Nam do môi...
3603,47280,Nghệ An có từ 14 đến 16 voi hoang dã (đứng thứ...,"Hoặc voi đực sống đơn lẻ ở khu vực xã Chi Khê,..."
3604,42151,Phi công Rudolf Erasmus hôm 7/4 cho biết anh đ...,Erasmus không tin nổi những gì đang xảy ra
...,...,...,...
4789,41870,Cảnh sát cho hay trách nhiệm truy tố và xét xử...,Theo truyền thông Tây Ban Nha thì hai công dân...
4790,47073,1. Thiết kế taxi bay an toàn và hiệu quả Sử dụ...,GEOS có thể phục vụ nghiên cứu hệ thống đại dư...
4791,46617,"Tối 1/6, ông Đỗ Huy Thông - Phó Chủ tịch UBND ...","Trong hành trình, du khách chỉ được ngắm vịnh ..."
4792,46410,"Các nhân viên đã rải bột mì để thử truy dấu ""k...",Họ không lắp đặt những camera đặc biệt được th...


In [ ]:
data['context_passage'] = data['context'].apply(preprocess_text)

In [ ]:
def predict_verdict(list_passage, claim):

    predicted_verdict = np.zeros((1,3))
    for passage in list_passage:
        input_ids = tokenizer.encode(passage, claim, return_tensors='pt')
        logits = nli_model(input_ids)[0]

        #0: contradiction, 1: neutral, 2: entailment
        #0: "SUPPORTED"  , 1:"NEI"   , 2:"REFUTED"
        probs = logits.softmax(dim=1).detach().cpu().numpy()
        predicted_verdict = np.concatenate((predicted_verdict, probs*100), axis = 0)

    #print(predicted_verdict)
    verdict, maxVerPassage = "NEI",""
    mxSim = 50
    for ver in [0,2]:
      for i in range(len(predicted_verdict)):
          if (predicted_verdict[i,ver] >= mxSim):
            verdict = ver
            maxVerPassage = list_passage[i-1]
            mxSim = predicted_verdict[i,ver]

    if (verdict == 0):
      verdict = "SUPPORTED"
    elif verdict == 2:
      verdict = "REFUTED"

    return (verdict, maxVerPassage)
predict_verdict(data['context_passage'][3600], data['claim'][3600])

('SUPPORTED',
 'năm 2015, sau khi tốt nghiệp trường thpt chuyên lê hồng phong, tp hcm, võ nguyên an, sinh năm 1997, quyết định "gap year" một năm để chuẩn bị hồ sơ, tiếng anh và du lịch trước khi đi du học.ước mơ của an là đến nước mỹ học ngành cơ khí, theo đuổi sở thích lắp ráp ôtô.tháng 4/2016, an trúng tuyển đại học virginia tech ở blacksburg, bang virginia.trường không cấp học bổng nên bố mẹ phải chu cấp cho an mỗi năm khoảng 45:000 usd (hơn 1 tỷ đồng) để con trai theo đuổi đam mê.')

In [ ]:
# for i in range(20):
#   print(predict_verdict(data['context_passage'][i], data['claim'][i]),  data['verdict'][i])

In [ ]:
def apply_predict_verdict(row):
    verdict, maxVerdictPassage = predict_verdict(row['context_passage'], row['claim'])
    return pd.Series([verdict, maxVerdictPassage])

In [ ]:
data[['verdict', 'maxVerdictPassage']] = data.apply(apply_predict_verdict, axis=1)
data.to_csv("/content/drive/MyDrive/Colab Notebooks/public_test1.csv")

OSError: ignored

In [ ]:
data.to_csv("/content/drive/MyDrive/Quân/public_test1.csv")

In [ ]:
from transformers import AutoModelForQuestionAnswering
import torch
import pandas as pd
from transformers import AutoTokenizer

# Tải mô hình từ đường dẫn đã lưu
model_checkpoint =  "Fsoft-AIC/videberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model_path = '/content/drive/MyDrive/Colab Notebooks/Quân/finetune MRC/videberta-base-finetuned 90.43 (max_len = 150)'
model = AutoModelForQuestionAnswering.from_pretrained(model_path)

# Chuyển model sang chế độ đánh giá (evaluation mode)
model.eval()
data= pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Quân/public_test_1.csv')
#data_warmup = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/warmup.csv')
data.head(5)

,Unnamed: 0.1,Unnamed: 0,index,context,claim,context_passage,verdict,maxVerdictPassage
0,0,0,29706,"Ngày 13/3, PGS.TS Phạm Cẩm Phương, Giám đốc Tr...",Nên kiểm tra gene trẻ sơ sinh khi gia đình có ...,"['ngày 13/3, pgs.ts phạm cẩm phương, giám đốc ...",NEI,NaN
1,1,1,32454,"STEM bao gồm lĩnh vực khoa học (Science), công...","mba tập trung vào kỹ năng kinh doanh, quản lý ...","['stem bao gồm lĩnh vực khoa học (science), cô...",NEI,NaN
2,2,2,41947,Trong khi tranh cãi còn chưa dứt về số tài liệ...,Cuộc tranh cãi còn chưa dứt về số tài liệu mật...,['trong khi tranh cãi còn chưa dứt về số tài l...,SUPPORTED,trong khi tranh cãi còn chưa dứt về số tài liệ...
3,3,3,47909,"Nằm trên đường Nguyễn Biểu, quận 5, tiệm há cả...","Tiệm há cảo của anh Hùng có 6 vị nhân, được là...","['nằm trên đường nguyễn biểu, quận 5, tiệm há ...",REFUTED,"tiệm há cảo của anh hùng có 8 vị nhân, được là..."
4,4,4,46798,"Năm nay, thời tiết nắng nhiều nhưng mưa ít khi...","Cách trung tâm Thủ đô chừng 100km, công viên T...","['năm nay, thời tiết nắng nhiều nhưng mưa ít k...",REFUTED,do nằm sâu trong ngõ nên đầm sen xuân đỉnh đượ...


In [ ]:
def getEvidence(question, context, verdict):
        # Tiền xử lý dữ liệu đầu vào
    if verdict == "NEI":
      return ''

    inputs = tokenizer(question, context, return_tensors="pt", padding=True, truncation=True)

        # Thực hiện dự đoán trả lời câu hỏi
    with torch.no_grad():
          outputs = model(**inputs)

        # Trích xuất kết quả từ outputs
    start_logits = outputs.start_logits
    end_logits = outputs.end_logits

      # Chuyển kết quả thành chỉ số câu trả lời tại vị trí có xác suất cao nhất
    start_index = torch.argmax(start_logits)
    end_index = torch.argmax(end_logits)

        # Lấy câu trả lời dựa trên chỉ số
    answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(inputs["input_ids"][0][start_index:end_index+1]))

    #lấy câu đầu
    return answer

In [ ]:
data.head(3)

,Unnamed: 0.1,Unnamed: 0,index,context,claim,context_passage,verdict,maxVerdictPassage,Evidence
0,0,0,29706,"Ngày 13/3, PGS.TS Phạm Cẩm Phương, Giám đốc Tr...",Nên kiểm tra gene trẻ sơ sinh khi gia đình có ...,"['ngày 13/3, pgs.ts phạm cẩm phương, giám đốc ...",NEI,NaN,
1,1,1,32454,"STEM bao gồm lĩnh vực khoa học (Science), công...","mba tập trung vào kỹ năng kinh doanh, quản lý ...","['stem bao gồm lĩnh vực khoa học (science), cô...",NEI,NaN,
2,2,2,41947,Trong khi tranh cãi còn chưa dứt về số tài liệ...,Cuộc tranh cãi còn chưa dứt về số tài liệu mật...,['trong khi tranh cãi còn chưa dứt về số tài l...,SUPPORTED,trong khi tranh cãi còn chưa dứt về số tài liệ...,trong khi tranh cãi còn chưa dứt về số tài liệ...


In [ ]:
data['Evidence'] = data.apply(lambda row: getEvidence(row['claim'], row['maxVerdictPassage'], row['verdict']), axis = 1)

In [ ]:
data.to_csv("/content/drive/MyDrive/Colab Notebooks/public_tes2.csv")

In [ ]:
data.head(3)

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,index,context,claim,context_passage,verdict,maxVerdictPassage,Evidence
0,0,0,0,29706,"Ngày 13/3, PGS.TS Phạm Cẩm Phương, Giám đốc Tr...",Nên kiểm tra gene trẻ sơ sinh khi gia đình có ...,"['ngày 13/3, pgs.ts phạm cẩm phương, giám đốc ...",NEI,NaN,NaN
1,1,1,1,32454,"STEM bao gồm lĩnh vực khoa học (Science), công...","mba tập trung vào kỹ năng kinh doanh, quản lý ...","['stem bao gồm lĩnh vực khoa học (science), cô...",NEI,NaN,NaN
2,2,2,2,41947,Trong khi tranh cãi còn chưa dứt về số tài liệ...,Cuộc tranh cãi còn chưa dứt về số tài liệu mật...,['trong khi tranh cãi còn chưa dứt về số tài l...,SUPPORTED,trong khi tranh cãi còn chưa dứt về số tài liệ...,trong khi tranh cãi còn chưa dứt về số tài liệ...


In [7]:
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
# Đọc tệp CSV
data1 = pd.read_csv('/content/drive/MyDrive/ColabNotebooks/Quân/submit/private_ANHMINH(:1500).csv')
data2 = pd.read_csv('/content/drive/MyDrive/ColabNotebooks/Quân/submit/private_test2(1500:3000).csv')
data3 = pd.read_csv('/content/drive/MyDrive/ColabNotebooks/Quân/submit/private_test_Danh(3000;4200).csv')
data4 = pd.read_csv('/content/drive/MyDrive/ColabNotebooks/Quân/submit/private_testQuan(4200:4800).csv')
data5 = pd.read_csv('/content/drive/MyDrive/ColabNotebooks/Quân/submit/private_testQuan(4800:).csv')

In [9]:
data = pd.concat([data1, data2, data3, data4, data5], axis=0)
data

,Unnamed: 0.1,Unnamed: 0,index,context,claim,context_passage,verdict,maxVerdictPassage,Evidence,Unnamed: 0.2
0,0,0,39773,"Sáng 7/6, bà Nguyễn Thị Lệ Thanh, Giám đốc Sở ...",Xuyên suốt thời gian diễn ra Festival biển Nha...,"['sáng 7/6, bà nguyễn thị lệ thanh, giám đốc s...",SUPPORTED,những thiết bị bay tham gia trình diễn các hìn...,"người đứng đầu sở du lịch khánh hòa cho hay, x...",NaN
1,1,1,37657,"Còn với Công nghệ dệt may, em không biết may h...",Một số người khuyên Nhi Pham theo Công nghệ th...,"['còn với công nghệ dệt may, em không biết may...",SUPPORTED,"còn với công nghệ dệt may, em không biết may h...",một số người khuyên em theo công nghệ thực phẩ...,NaN
2,2,2,40265,Nhà nghiên cứu Guangming Tao ở Đại học Khoa họ...,Nhiệt độ da bên dưới lớp cotton cao làm người ...,['nhà nghiên cứu guangming tao ở đại học khoa ...,REFUTED,loại vải này cũng có thể nhuộm theo nhiều màu ...,hsu cho rằng việc ứng dụng rộng rãi những loại...,NaN
3,3,3,47711,"Trong không gian của du thuyền 5 sao, Shynh Ho...",Nhãn hàng sẽ cùng 6 đối tác đề ra những chiến ...,"['trong không gian của du thuyền 5 sao, shynh ...",NEI,NaN,NaN,NaN
4,4,4,39930,Pin sinh học là giải pháp mới để cung cấp năng...,Bụi thông minh chỉ cần 5 tế bào vi khuẩn cũng ...,['pin sinh học là giải pháp mới để cung cấp nă...,NEI,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
591,591,5391,37369,"Mary Kate Keyes, Giám đốc dinh dưỡng của ứng d...","""Lúc đó, tôi đã quyết định về nhà vì tôi cảm t...","['mary kate keyes, giám đốc dinh dưỡng của ứng...",REFUTED,"mary kate keyes, giám đốc dinh dưỡng của ứng d...","""lúc đó, tôi thậm chí còn toát mồ hôi khi tự đ...",NaN
592,592,5392,37108,Cô gái Việt đang là nghiên cứu sinh tiến sĩ nă...,Ứng viên có thể thông qua mạng lưới bạn bè hoặ...,['cô gái việt đang là nghiên cứu sinh tiến sĩ ...,SUPPORTED,ứng viên có thể thông qua mạng lưới bạn bè hoặ...,ứng viên có thể thông qua mạng lưới bạn bè hoặ...,NaN
593,593,5393,37834,Người bệnh cho biết khoảng một tháng nay ăn uố...,"Tuy nhiên, chỉ số đường huyết của bệnh nhân lú...",['người bệnh cho biết khoảng một tháng nay ăn ...,REFUTED,"tuy nhiên, chỉ số đường huyết của bệnh nhân lú...","tuy nhiên, chỉ số đường huyết của bệnh nhân lú...",NaN
594,594,5394,43741,Cơ quan Giáo dục New Zealand (ENZ) hôm 5/1 cho...,Cơ quan Giáo dục New Zealand (ENZ) hôm 5/1 cho...,['cơ quan giáo dục new zealand (enz) hôm 5/1 c...,SUPPORTED,cơ quan giáo dục new zealand (enz) hôm 5/1 cho...,cơ quan giáo dục new zealand (enz) hôm 5/1 cho...,NaN


In [10]:
data.to_csv('/content/drive/MyDrive/ColabNotebooks/Quân/submit/private_test_final_verdict22.csv')

In [13]:
data = pd.read_csv('/content/drive/MyDrive/ColabNotebooks/Quân/submit/private_test_final_verdict22.csv')
data.head(10)

,Unnamed: 0.3,Unnamed: 0.1,Unnamed: 0,index,context,claim,context_passage,verdict,maxVerdictPassage,Evidence,Unnamed: 0.2
0,0,0,0,39773,"Sáng 7/6, bà Nguyễn Thị Lệ Thanh, Giám đốc Sở ...",Xuyên suốt thời gian diễn ra Festival biển Nha...,"['sáng 7/6, bà nguyễn thị lệ thanh, giám đốc s...",SUPPORTED,những thiết bị bay tham gia trình diễn các hìn...,"người đứng đầu sở du lịch khánh hòa cho hay, x...",NaN
1,1,1,1,37657,"Còn với Công nghệ dệt may, em không biết may h...",Một số người khuyên Nhi Pham theo Công nghệ th...,"['còn với công nghệ dệt may, em không biết may...",SUPPORTED,"còn với công nghệ dệt may, em không biết may h...",một số người khuyên em theo công nghệ thực phẩ...,NaN
2,2,2,2,40265,Nhà nghiên cứu Guangming Tao ở Đại học Khoa họ...,Nhiệt độ da bên dưới lớp cotton cao làm người ...,['nhà nghiên cứu guangming tao ở đại học khoa ...,REFUTED,loại vải này cũng có thể nhuộm theo nhiều màu ...,hsu cho rằng việc ứng dụng rộng rãi những loại...,NaN
3,3,3,3,47711,"Trong không gian của du thuyền 5 sao, Shynh Ho...",Nhãn hàng sẽ cùng 6 đối tác đề ra những chiến ...,"['trong không gian của du thuyền 5 sao, shynh ...",NEI,NaN,NaN,NaN
4,4,4,4,39930,Pin sinh học là giải pháp mới để cung cấp năng...,Bụi thông minh chỉ cần 5 tế bào vi khuẩn cũng ...,['pin sinh học là giải pháp mới để cung cấp nă...,NEI,NaN,NaN,NaN
5,5,5,5,37208,"Thạc sĩ, bác sĩ Lê Thị Hồng Thắm, khoa Hô hấp ...","Người mắc bệnh tim mạch, thắt cơ tim, dãn tim,...","['thạc sĩ, bác sĩ lê thị hồng thắm, khoa hô hấ...",SUPPORTED,"người mắc bệnh phổi tắc nghẽn mạn tính, hen su...","người mắc bệnh phổi tắc nghẽn mạn tính, hen su...",NaN
6,6,6,6,38194,Liên kết Dữ liệu An toàn Vaccine thuộc CDC Mỹ ...,CDC chỉ khuyến nghị trẻ em mắc Covid-19 vẫn p...,['liên kết dữ liệu an toàn vaccine thuộc cdc m...,REFUTED,"tiến sĩ william schaffner, chuyên gia về bệnh ...","cdc khuyến nghị người trưởng thành, người già ...",NaN
7,7,7,7,42930,Sự kiện cũng là không gian để chủ doanh nghiệp...,"Bên cạnh đó, HIU cũng công bố chương trình ""Họ...",['sự kiện cũng là không gian để chủ doanh nghi...,SUPPORTED,sự kiện quy tụ hàng trăm đơn vị lớn như ngân h...,"ngoài ra, hiu cũng công bố chương trình ""học b...",NaN
8,8,8,8,40100,Hãy tưởng tượng thời tiết mùa hè ở bang Louisi...,Mỗi thành viên trong đội tuần tra xoáy nước ấm...,['hãy tưởng tượng thời tiết mùa hè ở bang loui...,REFUTED,một nghiên cứu gần đây của viện nghiên cứu độc...,"khi bong bóng nổi lên, chúng mang theo nước lạ...",NaN
9,9,9,9,44099,"Theo ông Duy, mức thu 60 triệu đồng mỗi tháng ...",Cảng hoàn thành năm 2015,"['theo ông duy, mức thu 60 triệu đồng mỗi thán...",REFUTED,"nước ngọt dùng rửa tàu, sinh hoạt công ty cấp ...","cảng hoàn thành năm 2012, công suất thiết kế 2...",NaN


In [14]:
def post_preprocess(data):
    if data['verdict'] == 'NEI':
        data['Evidence'] = ''
        return data
    try:
        if data['Evidence'].count('.') > 0:
             data['Evidence'] = data['maxVerdictPassage'].split('.')[0]
            #print(2, data['Evidence'], type(data['Evidence']), 'xxx', data['Evidence'].count('.') > 0)
    except AttributeError:
         data['Evidence'] = data['maxVerdictPassage'].split('.')[0]
    except:
        print(20)

    return data

In [15]:
data = data.apply(lambda row: post_preprocess(row), axis = 1)

In [16]:
data.head(10)

,Unnamed: 0.3,Unnamed: 0.1,Unnamed: 0,index,context,claim,context_passage,verdict,maxVerdictPassage,Evidence,Unnamed: 0.2
0,0,0,0,39773,"Sáng 7/6, bà Nguyễn Thị Lệ Thanh, Giám đốc Sở ...",Xuyên suốt thời gian diễn ra Festival biển Nha...,"['sáng 7/6, bà nguyễn thị lệ thanh, giám đốc s...",SUPPORTED,những thiết bị bay tham gia trình diễn các hìn...,những thiết bị bay tham gia trình diễn các hìn...,NaN
1,1,1,1,37657,"Còn với Công nghệ dệt may, em không biết may h...",Một số người khuyên Nhi Pham theo Công nghệ th...,"['còn với công nghệ dệt may, em không biết may...",SUPPORTED,"còn với công nghệ dệt may, em không biết may h...",một số người khuyên em theo công nghệ thực phẩ...,NaN
2,2,2,2,40265,Nhà nghiên cứu Guangming Tao ở Đại học Khoa họ...,Nhiệt độ da bên dưới lớp cotton cao làm người ...,['nhà nghiên cứu guangming tao ở đại học khoa ...,REFUTED,loại vải này cũng có thể nhuộm theo nhiều màu ...,hsu cho rằng việc ứng dụng rộng rãi những loại...,NaN
3,3,3,3,47711,"Trong không gian của du thuyền 5 sao, Shynh Ho...",Nhãn hàng sẽ cùng 6 đối tác đề ra những chiến ...,"['trong không gian của du thuyền 5 sao, shynh ...",NEI,NaN,,NaN
4,4,4,4,39930,Pin sinh học là giải pháp mới để cung cấp năng...,Bụi thông minh chỉ cần 5 tế bào vi khuẩn cũng ...,['pin sinh học là giải pháp mới để cung cấp nă...,NEI,NaN,,NaN
5,5,5,5,37208,"Thạc sĩ, bác sĩ Lê Thị Hồng Thắm, khoa Hô hấp ...","Người mắc bệnh tim mạch, thắt cơ tim, dãn tim,...","['thạc sĩ, bác sĩ lê thị hồng thắm, khoa hô hấ...",SUPPORTED,"người mắc bệnh phổi tắc nghẽn mạn tính, hen su...","người mắc bệnh phổi tắc nghẽn mạn tính, hen su...",NaN
6,6,6,6,38194,Liên kết Dữ liệu An toàn Vaccine thuộc CDC Mỹ ...,CDC chỉ khuyến nghị trẻ em mắc Covid-19 vẫn p...,['liên kết dữ liệu an toàn vaccine thuộc cdc m...,REFUTED,"tiến sĩ william schaffner, chuyên gia về bệnh ...","tiến sĩ william schaffner, chuyên gia về bệnh ...",NaN
7,7,7,7,42930,Sự kiện cũng là không gian để chủ doanh nghiệp...,"Bên cạnh đó, HIU cũng công bố chương trình ""Họ...",['sự kiện cũng là không gian để chủ doanh nghi...,SUPPORTED,sự kiện quy tụ hàng trăm đơn vị lớn như ngân h...,sự kiện quy tụ hàng trăm đơn vị lớn như ngân h...,NaN
8,8,8,8,40100,Hãy tưởng tượng thời tiết mùa hè ở bang Louisi...,Mỗi thành viên trong đội tuần tra xoáy nước ấm...,['hãy tưởng tượng thời tiết mùa hè ở bang loui...,REFUTED,một nghiên cứu gần đây của viện nghiên cứu độc...,một nghiên cứu gần đây của viện nghiên cứu độc...,NaN
9,9,9,9,44099,"Theo ông Duy, mức thu 60 triệu đồng mỗi tháng ...",Cảng hoàn thành năm 2015,"['theo ông duy, mức thu 60 triệu đồng mỗi thán...",REFUTED,"nước ngọt dùng rửa tàu, sinh hoạt công ty cấp ...","cảng hoàn thành năm 2012, công suất thiết kế 2...",NaN


In [17]:
submit_data = data[['index','Evidence', 'verdict']]
submit_data

,index,Evidence,verdict
0,39773,những thiết bị bay tham gia trình diễn các hìn...,SUPPORTED
1,37657,một số người khuyên em theo công nghệ thực phẩ...,SUPPORTED
2,40265,hsu cho rằng việc ứng dụng rộng rãi những loại...,REFUTED
3,47711,,NEI
4,39930,,NEI
...,...,...,...
5391,37369,"""lúc đó, tôi thậm chí còn toát mồ hôi khi tự đ...",REFUTED
5392,37108,ứng viên có thể thông qua mạng lưới bạn bè hoặ...,SUPPORTED
5393,37834,"tuy nhiên, chỉ số đường huyết của bệnh nhân lú...",REFUTED
5394,43741,cơ quan giáo dục new zealand (enz) hôm 5/1 cho...,SUPPORTED


In [18]:
import json

# Tạo một empty dictionary
data_dict = {}

# Lặp qua từng dòng của DataFrame
for i, row in submit_data.iterrows():
    data_dict[row['index']] = {
        'verdict': row['verdict'],
        'evidence': row['Evidence']
    }

# Lưu dictionary vào tệp JSON
with open('/content/drive/MyDrive/ColabNotebooks/Quân/submit/private_result.json', 'w') as json_file:
    json.dump(data_dict, json_file, ensure_ascii=False, indent=4)